## LangChain-QwQ 相关API使用代码

## 接入QwQ模型

最开始LangChain-QwQ是用于链接QwQ模型，但是随着后续的改进,从v0.2.0版本，已经支持了大部分阿里云百炼的Qwen系列模型，包括Qwen3系列模型。

支持的模型列表参考以下两个文章：
**支持openai风格的百炼大模型**：https://help.aliyun.com/zh/model-studio/compatibility-of-openai-with-dashscope?spm=a2c4g.11186623.help-menu-2400256.d_2_9_0.4f222aa15hTLmr
**支持openai风格的百炼视觉大模型**：https://help.aliyun.com/zh/model-studio/qwen-vl-compatible-with-openai?spm=a2c4g.11186623.help-menu-2400256.d_2_9_2.73b94c0ctVvkrd&scm=20140722.H_2845564._.OR_help-T_cn~zh-V_1

**总结**：因为这个库是采用openai sdk的方式（继承langchain-openai)，因此只要这个Qwen模型支持openai风格，理论上都可以接入，因此可以接入第三方的Qwen模型，例如硅基流动或者本地大模型(自己部署)
支持QwQ以外的其它模型的主要原因是因为**langchain-community**的**ChatTongyi**尚有bug，且功能支持度有待加强，所以换成LangChain-QwQ来支持。


确保你安装了langchain-qwq的最新版

In [1]:
import langchain_qwq
langchain_qwq.__version__

'0.2.0'

### 导入类

In [2]:

from langchain_qwq import ChatQwQ
from langchain_qwq.utils import convert_reasoning_to_content,aconvert_reasoning_to_content
from langchain_core.messages import AIMessageChunk,AIMessage

### 简单调用

在此之前需要设置环境变量

分别需要设置 DASHSCOPE_API_KEY 和 DASHSCOPE_API_BASE 环境变量。这是因为默认下langchain-qwq用的是国际版的阿里云，而国内开发者一般使用国内的阿里云

In [3]:
model=ChatQwQ(
    model="qwq-32b"
)

In [4]:
model.invoke("你好")

AIMessage(content='你好！有什么我可以帮你的吗？', additional_kwargs={'reasoning_content': '好的，用户发来了“你好”，我需要用中文回复。首先，我要确保回应友好，符合中文的问候习惯。然后根据之前的设定，要保持口语化和简洁，避免使用复杂结构。\n\n用户可能只是打招呼，或者有后续的问题。我需要准备好接下来的对话，所以回复要留有空间让用户继续表达。比如用“有什么我可以帮你的吗？”来引导他们说明需求。\n\n同时，要检查有没有需要特别注意的地方，比如用户之前是否有过对话历史，但看起来这是新对话，所以不需要回顾之前的交流。情绪方面，保持中立和友好，符合角色设定。\n\n还要确保不使用任何Markdown格式，保持纯文本。现在组织语言，确保自然流畅，没有语法错误。最后确认回复符合所有要求：口语化、简洁、有帮助，并且推动对话进展。'}, response_metadata={'model_name': 'qwq-32b'}, id='run--f1cf46bb-e70c-4ea0-8d01-d3f314758a55-0', usage_metadata={'input_tokens': 9, 'output_tokens': 184, 'total_tokens': 193, 'input_token_details': {}, 'output_token_details': {}})

### 访问思考内容

In [5]:
response=model.invoke("你好")
response.additional_kwargs["reasoning_content"]

'好的，用户发来“你好”，我需要回应。首先，要友好地打招呼，可能用“你好！有什么我可以帮你的吗？”这样既回应了问候，又询问需求。接着，根据用户之前的指示，要保持口语化和简洁，所以不用太复杂。然后，检查是否有需要考虑的其他因素，比如用户是否有之前的历史对话，但这里看起来是第一次互动，所以直接按基础回复处理。还要确保符合所有规则，比如不使用Markdown格式，保持自然。最后，确认回复能推动对话进展，让用户知道可以提出具体问题或请求。没问题，这样回复应该合适。'

### 异步调用

In [6]:
await model.ainvoke("你好")

AIMessage(content='你好！有什么我可以帮你的吗？', additional_kwargs={'reasoning_content': '好的，用户发来了“你好”，我需要用中文回复。首先，我要确认用户的需求，可能只是打个招呼，或者有进一步的问题。根据之前的设定，我的回复要口语化、简洁，同时保持友好和开放。我应该回应问候，并邀请他们提出问题或需求。还要注意不要使用任何格式，保持自然。接下来，检查是否有需要调整的地方，比如是否足够亲切，有没有可能用户想讨论的话题。暂时没有更多信息，所以保持简单最好。确定回复：“你好！有什么我可以帮你的吗？” 这样既回应了问候，又鼓励用户进一步交流。'}, response_metadata={'model_name': 'qwq-32b'}, id='run--990b90f4-0d4a-4a85-9e06-b66ffe949750-0', usage_metadata={'input_tokens': 9, 'output_tokens': 137, 'total_tokens': 146, 'input_token_details': {}, 'output_token_details': {}})

### 流式输出

In [7]:
isfirst,isend=True,True
for msg in model.stream("你好"):
    if (
        isinstance(msg, AIMessageChunk)
        and "reasoning_content" in msg.additional_kwargs
    ):
        if isfirst:
            print("开始思考")
            isfirst = False   
        print(msg.additional_kwargs["reasoning_content"],end="",flush=True)
    elif (
        isinstance(msg, AIMessageChunk)
        and "reasoning_content" not in msg.additional_kwargs
        and msg.content
    ):
        if isend:
            print()
            print("思考结束")
            isend = False
        else:
            print(msg.content,end="",flush=True)





开始思考
好的，用户发来了“你好”，我需要回应。首先，根据之前的设定，我应该保持口语化和简洁。用户可能只是打招呼，或者想开始对话。我需要友好回应，同时引导他们进一步说明需求。可能用户接下来会问问题或寻求帮助，所以我的回复要开放且鼓励他们继续。注意不要用复杂结构，保持自然。比如可以说“你好！有什么我可以帮你的吗？”这样既回应了问候，又主动提供帮助，符合角色设定。同时，检查是否有其他潜在需求，但用户现在可能还没具体说明，所以保持简单即可。避免使用表情符号或过多修饰，保持专业又亲切的语气。
思考结束
！有什么我可以帮你的吗？

### 异步流式输出

In [8]:
isfirst=True
isend=True
async for msg in model.astream("你好"):
    if (
        isinstance(msg, AIMessageChunk)
        and "reasoning_content" in msg.additional_kwargs
    ):
        if isfirst:
            print("开始思考")
            isfirst = False
        print(msg.additional_kwargs["reasoning_content"],end="",flush=True)
    elif (
        isinstance(msg, AIMessageChunk)
        and "reasoning_content" not in msg.additional_kwargs
        and msg.content
    ):
        if isend:   
            print("\n")
            print("思考结束")
            isend = False
        else:
            print(msg.content,end="",flush=True)

开始思考
嗯，用户发来“你好”，我应该怎么回应呢？首先，我需要确定用户的意图。可能他们只是打招呼，或者想测试我的反应。根据之前的指示，我应该友好回应，并保持口语化。

接下来，我要确保回应简洁，同时提供帮助的意愿。比如可以说“你好！有什么我可以帮你的吗？”这样既回应了问候，又引导用户提出需求。不过，可能需要更自然一些，避免太机械。

另外，用户可能有不同的需求，比如需要信息、解决问题，或者只是随便聊聊。我需要保持开放，让用户决定接下来的方向。有没有必要加入表情符号或更亲切的语言？比如加上“😊”可能会显得更友好，但根据之前的例子，可能不需要。

还要注意不要过于复杂，保持简洁。比如“你好！今天过得怎么样？”可能会让对话继续下去，但用户可能直接想进入问题。所以最好先确认他们需要什么帮助，再进一步行动。

总结一下，最佳回应应该是友好、简洁，并主动提供帮助。例如：“你好！有什么我可以帮你的吗？”这样既符合要求，又有效引导用户说明需求。

思考结束
我可以帮你的吗？😊

上述的便捷方式如下  

In [9]:
for msg in convert_reasoning_to_content(model.stream("你好")):
    print(msg.content,end="",flush=True)

<think>好的，用户发来了“你好”，我需要用中文回复。首先，我要保持友好和热情，回应问候。然后，按照用户的要求，提供帮助，询问有什么可以帮到他们。同时，要确保回复口语化，简洁易懂，避免使用复杂句子。还要注意不要使用Markdown格式，保持自然的中文表达。现在检查一下是否符合所有要求：友好、简洁、口语化，没有格式问题。没问题，可以回复了。</think>你好！今天过得怎么样？有什么我可以帮你的吗？

### 工具调用

In [10]:
from langchain_core.tools import tool

@tool
def get_weather(city:str)->str:
    """获取城市天气"""
    return f"{city}的天气是晴天。"



bind_model=model.bind_tools([get_weather])

bind_model.invoke("西安的天气如何").tool_calls #type:ignore


[{'name': 'get_weather',
  'args': {'city': '西安'},
  'id': 'call_6bf77c9018e44587836df7',
  'type': 'tool_call'}]

也可以支持并行工具调用

In [11]:
bind_model.invoke("西安和天津的天气如何",  parallel_tool_calls=True).tool_calls #type:ignore

[{'name': 'get_weather',
  'args': {'city': '西安'},
  'id': 'call_f62684a49255448888a2c3',
  'type': 'tool_call'},
 {'name': 'get_weather',
  'args': {'city': '天津'},
  'id': 'call_a2ab0ecf614643849bdbb8',
  'type': 'tool_call'}]

#### 结构化输出

支持function calling和json mode

In [12]:
from pydantic import BaseModel

In [13]:
class User(BaseModel):
    name:str
    age:int

In [14]:
struct_model=model.with_structured_output(User,method="json_mode") 

In [15]:
struct_model.invoke("你好，我叫张三，今年21岁")

User(name='张三', age=21)

In [16]:
struct_model=model.with_structured_output(User,method="function_calling")

In [17]:
struct_model.invoke("你好，我叫张三")

### 与LangChain的结合使用

In [18]:
from langchain.agents import create_tool_calling_agent,AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

In [19]:
agent=create_tool_calling_agent(
    model,
    [get_weather],
    prompt=ChatPromptTemplate.from_messages(
                [
                    ("system", "You are a helpful assistant"),
                    ("placeholder", "{chat_history}"),
                    ("human", "{input}"),
                    ("placeholder", "{agent_scratchpad}"),
                ]
            )
)

In [20]:
agent_executor=AgentExecutor(agent=agent,tools=[get_weather])
agent_executor.invoke({"input":"西安的天气如何"})

{'input': '西安的天气如何', 'output': '您好！西安今天的天气是晴天。'}

## 接入其它Qwen模型

langchain-qwq虽然常用于接入QwQ模型，但是不仅仅只能接入QwQ模型，这个包提供了ChatQwen，这个类可以接入大部分的Qwen系列模型（包括开源和闭源），最重要的是这个类对Qwen3模型做了额外的参数支持，包括思考功能。












### 接入Qwen3模型

使用方式和接入QwQ一样，因此这里就简单说一下用法

In [21]:
from langchain_qwq import ChatQwen

In [22]:
model=ChatQwen(model="qwen3-235b-a22b") #也可以用别的模型

#### 普通的调用

In [23]:
model.invoke("你好")

AIMessage(content='你好呀！✨ 很高兴见到你！今天过得怎么样呀？希望你度过了愉快的一天。我随时准备好陪你聊天、帮你解决问题，或者就这样轻松愉快地闲聊一会儿。有什么想跟我分享的吗？ 🌟', additional_kwargs={'reasoning_content': '用户跟我打了一个简单的招呼“你好”。这是一个友好的开场白，需要我以温暖亲切的态度回应。在日常社交中，这样的问候往往代表着想要开启一段对话的意愿。我应该用同样友好的方式回应，并主动引导对话继续下去。同时要注意保持自然的语气，避免过于正式或生硬。可以想象这是一个朋友间的日常寒暄，所以回应要轻松愉快。我需要为接下来可能的各种对话场景做好准备，无论是闲聊还是讨论正事都可以自然承接。回应中加入一些开放性的问题或建议，能够给对方更多回应的选择，有助于对话继续深入进行。保持积极乐观的态度，展现出乐于交流的意愿。'}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen3-235b-a22b'}, id='run--3e1fda44-8ee0-43af-86db-8085ef7130fe-0', usage_metadata={'input_tokens': 9, 'output_tokens': 195, 'total_tokens': 204, 'input_token_details': {}, 'output_token_details': {'reasoning': 139}})

#### 异步调用

In [24]:
await model.ainvoke("你好")

AIMessage(content='你好！有什么我可以帮助你的吗？', additional_kwargs={'reasoning_content': '嗯，用户发来了“你好”，这是一个很常见的中文问候语。我需要回应得友好且自然。首先，我应该用中文回复，因为用户用了中文打招呼。然后，保持简洁但亲切，比如“你好！有什么我可以帮助你的吗？”这样既回应了问候，又主动询问是否需要帮助，符合聊天助手的角色。另外，要注意语气不要太机械，带点热情。可能用户接下来会问问题，或者需要进一步的交流，所以这个回复也为后续对话打开了空间。不需要使用复杂的句子结构，保持口语化。确认没有拼写错误，标点符号正确。好的，这样的回复应该合适。'}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen3-235b-a22b'}, id='run--f334a43a-5168-48fc-938d-8ec2d49d68da-0', usage_metadata={'input_tokens': 9, 'output_tokens': 145, 'total_tokens': 154, 'input_token_details': {}, 'output_token_details': {'reasoning': 132}})

#### 关闭思考模式

这个仅对Qwen3系列（开源或者闭源模型有效），阿里云文档表明，对于开源模型默认开启深度思考模式，而闭源模型则是关闭

In [25]:
disthink_model=ChatQwen(model="qwen3-235b-a22b",enable_thinking=False)

In [26]:
disthink_model.invoke("你好")

AIMessage(content='你好呀！✨ 很高兴见到你！今天过得怎么样呀？希望你度过了愉快的一天。我随时准备好陪你聊天、帮你解决问题，或者就这样轻松愉快地闲聊一会儿。有什么想跟我分享的吗？ 🌟', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen3-235b-a22b'}, id='run--754e98f7-607c-416b-9222-65157d501bbd-0', usage_metadata={'input_tokens': 13, 'output_tokens': 51, 'total_tokens': 64, 'input_token_details': {}, 'output_token_details': {}})

对于闭源模型，你如果要开启深度思考，则需要设置参数为True

In [27]:
think_model=ChatQwen(model="qwen-plus-latest",enable_thinking=True)

In [28]:
think_model.invoke("你好")

AIMessage(content='你好呀！有什么我可以帮助你的吗？😊', additional_kwargs={'reasoning_content': '嗯，用户发来一个“你好”，我应该怎么回应呢？首先，我需要确定用户的需求是什么。可能只是打个招呼，或者有后续的问题要问。作为AI助手，我的任务是友好且乐于助人。所以，回复应该既亲切又鼓励用户继续交流。让我想想，用中文的话，通常会说“你好呀！”或者“您好！有什么我可以帮助您的吗？”之类的。不过用户可能希望更口语化一点，所以用“你好呀！”可能更合适。然后加上一句询问是否需要帮助，这样可以引导用户提出具体的问题。另外，要注意保持积极和热情的语气，让用户感觉被重视。有没有必要加上表情符号呢？比如😊，这样可以让回复看起来更友好。不过要根据平台的规定，有些地方可能不建议使用。暂时先不加，保持简洁。然后检查有没有语法错误或者用词不当。看起来没问题。总结一下，回复应该是：你好呀！有什么我可以帮助你的吗？这样既友好又开放，让用户有继续对话的意愿。'}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen-plus-latest'}, id='run--c54a67f5-bd9d-4b52-834b-18a29d66500b-0', usage_metadata={'input_tokens': 9, 'output_tokens': 231, 'total_tokens': 240, 'input_token_details': {}, 'output_token_details': {'reasoning': 216}})

### 设置思考长度

对于Qwen3模型，还可以设置思考长度

In [29]:
think_model=ChatQwen(model="qwen3-235b-a22b",thinking_budget=20)

In [30]:
think_model.invoke("你好")

AIMessage(content='你好！有什么我可以帮你的吗？', additional_kwargs={'reasoning_content': '嗯，用户发来了一句“你好”，我应该怎么回应呢？首先，我需要确定用户的'}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen3-235b-a22b'}, id='run--3ae19290-26e8-4293-84cb-516b7f1b5c65-0', usage_metadata={'input_tokens': 9, 'output_tokens': 33, 'total_tokens': 42, 'input_token_details': {}, 'output_token_details': {'reasoning': 20}})

### 工具调用

In [31]:
bind_model=model.bind_tools([get_weather])

In [32]:
bind_model.invoke("查询西安的天气").tool_calls #type:ignore

[{'name': 'get_weather',
  'args': {'city': '西安'},
  'id': 'call_b3bcf1faaa8e40b2add69e',
  'type': 'tool_call'}]

In [33]:
(await bind_model.ainvoke("查询西安和天津的天气",parallel_tool_calls=True)).tool_calls #type:ignore

[{'name': 'get_weather',
  'args': {'city': '西安'},
  'id': 'call_4fa01ea67498434993da1d',
  'type': 'tool_call'},
 {'name': 'get_weather',
  'args': {'city': '天津'},
  'id': 'call_8cf74579cc11458facf945',
  'type': 'tool_call'}]

### 结构化输出

In [34]:
struct_model=model.with_structured_output(User,method="function_calling")
struct_model.invoke("我叫张三，今年21岁")

User(name='张三', age=21)

In [35]:
struct_model=model.with_structured_output(User,method="json_mode")
await struct_model.ainvoke("你好，我叫张三，今年21岁") #这个模式下会关闭思考模型不然会报错

User(name='张三', age=21)

###  接入其它系列的模型

例如 Qwen-Max

In [36]:
model=ChatQwen(model="qwen-max")
print(model.invoke("你好"))
print(model.bind_tools([get_weather]).invoke("查询西安和温州的天气",parallel_tool_calls=True).tool_calls) #type:ignore
print(model.with_structured_output(User,method="json_mode").invoke("你好，我叫张三，今年21岁"))

content='你好！有什么可以帮助你的吗？' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'qwen-max'} id='run--12b8e987-89ab-4f85-9380-381cb4773951-0' usage_metadata={'input_tokens': 9, 'output_tokens': 7, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}
[{'name': 'get_weather', 'args': {'city': '西安'}, 'id': 'call_017cb25cac5c4067984955', 'type': 'tool_call'}, {'name': 'get_weather', 'args': {'city': '温州'}, 'id': 'call_8bab2508a86d4af087f016', 'type': 'tool_call'}]
name='张三' age=21


或者是 Qwen2.5-72B-Instruct

In [37]:
model=ChatQwen(model="qwen2.5-72b-instruct")
print(model.invoke("你好"))
print(model.bind_tools([get_weather]).invoke("查询西安和温州的天气",parallel_tool_calls=True).tool_calls) #type:ignore
print(model.with_structured_output(User,method="json_mode").invoke("你好，我叫张三，今年21岁"))

content='你好！有什么可以帮助你的吗？' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2.5-72b-instruct'} id='run--b1cfdedb-610b-4a07-8930-8ded47504919-0' usage_metadata={'input_tokens': 9, 'output_tokens': 7, 'total_tokens': 16, 'input_token_details': {}, 'output_token_details': {}}
[{'name': 'get_weather', 'args': {'city': '西安'}, 'id': 'call_bad50f6569f14920aff83d', 'type': 'tool_call'}, {'name': 'get_weather', 'args': {'city': '温州'}, 'id': 'call_33e5896dfb604ba1af9b87', 'type': 'tool_call'}]
name='张三' age=21


甚至你可以接入视觉模型

In [38]:
from langchain_core.messages import HumanMessage

In [39]:
model=ChatQwen(model="qwen-vl-max-latest")
messages=[
    HumanMessage(content=[
        
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://help-static-aliyun-doc.aliyuncs.com/file-manage-files/zh-CN/20241022/emyrja/dog_and_girl.jpeg"
                    },
                },
                {"type": "text", "text": "图中描绘的是什么景象?"},
            
    ])
]
print(model.invoke(messages))

content='图中描绘的是一个温馨的海滩场景。具体细节如下：\n\n1. **人物和宠物**：画面中有一个人和一只狗。这个人坐在沙滩上，穿着格子衬衫和深色裤子，面带微笑，正在与狗互动。狗是一只金毛犬，戴着彩色的项圈，前爪抬起，似乎在和人“握手”。\n\n2. **环境**：背景是一个海滩，可以看到海浪轻轻拍打着岸边。沙滩上有细腻的沙粒，显得非常柔软。\n\n3. **光线**：照片拍摄的时间看起来是日落时分，阳光柔和，给整个画面增添了一种温暖的氛围。\n\n4. **情感**：整体画面给人一种宁静、愉快的感觉，展现了人与宠物之间的亲密关系和美好的时光。\n\n这个场景传达了人与自然和谐相处的美好瞬间。' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'qwen-vl-max-latest'} id='run--0207abf6-865c-4d47-8e9e-4aa253742126-0' usage_metadata={'input_tokens': 1264, 'output_tokens': 172, 'total_tokens': 1436, 'input_token_details': {}, 'output_token_details': {}}


## 接入第三方平台的Qwen模型

### 硅基流动

以Qwen3-32B为例，使用langchain-qwq进行连接调用

第一步 设置环境变量

In [40]:
import os
os.environ["DASHSCOPE_API_BASE"] = "https://api.siliconflow.cn/v1"
os.environ["DASHSCOPE_API_KEY"] = os.getenv("SILICONFLOW_API_KEY") or ""

In [41]:
model = ChatQwen(model="Qwen/Qwen3-32B")
await model.ainvoke("你好")

AIMessage(content='\n\n你好呀！今天想聊聊什么呢？或者有什么我可以帮忙的吗？😊', additional_kwargs={'reasoning_content': '\n嗯，用户发来了“你好”，这是很常见的开场白。我需要友好回应，同时保持自然和亲切。第一次交互关系用户会感觉轻松。由于是中文对话，可以融入一些口语化的表达，让交流更生动。考虑到对话的开放性，可以从问候开始，然后询问今日计划或有什么能帮忙的地方。轻松的语气可以让用户更容易敞开心扉。在日常问答场景里，简短且态度友好的回复往往是最有效的。比如用“早安/午安/晚安”+“你好呀！”开头，让每一次对话都从温暖的问候开始。然后要照顾到用户的不同需求，无论是寻求知识、娱乐还是情感支持。现在是上午时间，可以用对应的祝福语加上轻松的提问。用户可能想要一个友好而专业的对话伙伴。所以回复既要体现温暖亲切，又要保持开放和包容。\n\n通过问候和轻微的好奇心，引导用户表达更多。如果用户后续提出具体问题或需求，可以更精准地提供帮助。比如现在回复“你好呀！今天想聊聊什么呢？或者有什么我可以帮忙的吗？”这样的回复自然且具有互动性。保持愉悦的心情很重要，所以可以加入一些表情符号来增添亲切感。\n'}, response_metadata={'finish_reason': 'stop', 'model_name': 'Qwen/Qwen3-32B'}, id='run--c537f16b-7625-4564-8909-cbd1758b2589-0', usage_metadata={'input_tokens': 2457, 'output_tokens': 37125, 'total_tokens': 39582, 'input_token_details': {}, 'output_token_details': {'reasoning': 36938}})

In [42]:
async for msg in model.astream("你好"):
    print(msg.content,end="",flush=True)



你好！👋 有什么我可以帮你的吗？无论是想聊聊天，还是需要解答什么问题，我都很乐意帮忙哦。😊 让我知道你想聊些什么吧！

### 自部署开源大模型

这个要求有点高，首先你得有硬件，才能部署。这里用本地部署的qwen3-8b模型进行测试
首先需要利用vllm框架将其部署
需要明确的是**reasoning-parser**是不要填写Qwen文档说的deepseek_r1,这个会有bug，要填写最新的qwen3，当然这个需要比较高版本的vllm框架。比如0.9.0

In [43]:
os.environ["DASHSCOPE_API_BASE"]="http://localhost:8000/v1"
os.environ["DASHSCOPE_API_KEY"]="sk-1234567890"

In [44]:
model=ChatQwen(model="Qwen/Qwen3-8B")
await model.ainvoke("你好")

AIMessage(content='\n\n你好！😊 有什么我可以帮你的吗？无论是解答问题、提供建议，还是闲聊，我都很乐意和你交流！', additional_kwargs={'reasoning_content': '\n嗯，用户发来了一条“你好”，看起来是想开始一段对话。首先，我需要确认用户的意图，可能他们只是随便打个招呼，或者有具体的问题需要帮助。作为AI助手，我应该保持友好和开放的态度，回应他们的问候，并主动询问是否需要帮助。\n\n接下来，我需要考虑用户可能的背景。他们可能是第一次使用这个平台，或者之前有过互动。如果是第一次，可能需要更详细的引导；如果之前有交流，可能需要根据历史记录调整回应。不过当前情况下，没有历史记录，所以保持通用的友好回应比较合适。\n\n另外，用户可能有不同的需求，比如寻求信息、解决问题，或者只是闲聊。因此，我的回应应该既友好又具有引导性，鼓励他们进一步说明需求。同时，要注意语气亲切，避免过于机械化的回复，让用户感到被重视。\n\n还要注意文化差异和语言习惯，确保用词恰当，避免误解。比如“你好”在中文里是常见的问候语，回应时使用类似的表达会更自然。同时，提供帮助的选项，比如询问是否有具体问题，或者是否需要某个方面的信息，这样用户更容易表达他们的需求。\n\n最后，保持简洁明了，避免冗长的回复，让用户能够快速理解并回应。确保回复符合平台的规定和礼仪，不涉及敏感话题，保持中立和客观。总结来说，回应需要友好、开放、引导性，并且简洁明了，让用户感到舒适并愿意进一步交流。\n'}, response_metadata={'finish_reason': 'stop', 'model_name': 'Qwen/Qwen3-8B'}, id='run--e6537654-4a17-438c-99c4-a5bff277f439-0', usage_metadata={'input_tokens': 9, 'output_tokens': 339, 'total_tokens': 348, 'input_token_details': {}, 'output_token_details': {}})

In [45]:
bind_model=model.bind_tools([get_weather])
await bind_model.ainvoke("查询温州和西安的天气")

AIMessage(content='\n\n\n', additional_kwargs={'reasoning_content': '\n好的，用户需要查询温州和西安的天气。我有两个函数调用的机会。首先，我需要调用get_weather函数来获取温州的天气。然后，再调用一次get_weather函数来获取西安的天气。每个函数都需要传入相应的城市参数。确保两次调用的参数正确，然后返回两个结果。\n', 'tool_calls': [{'index': 0, 'id': 'chatcmpl-tool-4157997759fb438c8037a1f60238905c', 'function': {'arguments': '{"city": "温州"}', 'name': 'get_weather'}, 'type': 'function'}, {'index': 1, 'id': 'chatcmpl-tool-1b11d98485df4fb3a324d8013fbbe8ce', 'function': {'arguments': '{"city": "西安"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'Qwen/Qwen3-8B'}, id='run--a983d88f-cf19-4eef-83f5-650ec602f39b-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '温州'}, 'id': 'chatcmpl-tool-4157997759fb438c8037a1f60238905c', 'type': 'tool_call'}, {'name': 'get_weather', 'args': {'city': '西安'}, 'id': 'chatcmpl-tool-1b11d98485df4fb3a324d8013fbbe8ce', 'type': 'tool_call'}], usage_metadata={'input_tokens': 149, 'output_tokens': 114, 'total_

In [46]:
model=ChatQwen(model="Qwen/Qwen3-8B",extra_body={
    "chat_template_kwargs":{
        "enable_thinking":False
    }
})


In [47]:
await model.ainvoke("你好")

AIMessage(content='你好！😊 有什么我可以帮助你的吗？', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'Qwen/Qwen3-8B'}, id='run--13e09d68-d56f-4f89-b09f-fb44d92dd05f-0', usage_metadata={'input_tokens': 13, 'output_tokens': 11, 'total_tokens': 24, 'input_token_details': {}, 'output_token_details': {}})